In [5]:
import pandas as pd
import os
import numpy as np
from os import listdir
from os.path import isfile, join

In [29]:
#January 1st, 24 hours 
df = pd.read_csv("/Users/yukahatori/A_Fairness/USW00023234.csv")

#drop columns we don't need
df.drop('STATION', axis=1, inplace=True)
df.drop('NAME', axis=1, inplace=True)
df.drop('LATITUDE', axis=1, inplace=True)
df.drop('LONGITUDE', axis=1, inplace=True)
df.drop('ELEVATION', axis=1, inplace=True)
df.drop('month', axis=1, inplace=True)
df.drop('hour', axis=1, inplace=True)
df.drop('day', axis=1, inplace=True)

#checks the flags and removes any rows corresponding with 'M' which means the data is missing
measurementFlags = list(df.filter(regex='meas'))
for flag in measurementFlags:
    df = df[~df[flag].str.contains('M')]

completenessFlags = list(df.filter(regex='meas'))
for flag in completenessFlags:
    df = df[~df[flag].str.contains('-9999')]
        
#get rid of all measurement and completeness flags
df = df[df.columns.drop(list(df.filter(regex='meas')))]
df = df[df.columns.drop(list(df.filter(regex='comp')))]

#rename columns
df.columns = ['date', 'HLY_TEMP_NORMAL', 'years_HLY_TEMP_NORMAL', 'HLY_TEMP_10PCTL',
       'years_HLY_TEMP_10PCTL', 'HLY_TEMP_90PCTL', 'years_HLY_TEMP_90PCTL',
       'HLY_DEWP_NORMAL', 'years_HLY_DEWP_NORMAL', 'HLY_DEWP_10PCTL',
       'years_HLY_DEWPv10PCTL', 'HLY_DEWP_90PCTL', 'years_HLY_DEWP_90PCTL',
       'HLY_PRES_NORMAL', 'years_HLY_PRES_NORMAL', 'HLY_PRES_10PCTL',
       'years_HLY_PRES_10PCTL', 'HLY_PRES_90PCTL', 'years_HLY_PRES_90PCTL',
       'HLY_CLDH_NORMAL', 'years_HLY_CLDH_NORMAL', 'HLY_HTDH_NORMAL',
       'years_HLY_HTDH_NORMAL', 'HLY_CLOD_PCTCLR', 'years_HLY_CLOD_PCTCLR',
       'HLY_CLOD_PCTFEW', 'years_HLY_CLOD_PCTFEW', 'HLY_CLOD_PCTSCT',
       'years_HLY_CLOD_PCTSCT', 'HLY_CLOD_PCTBKN', 'years_HLY_CLOD_PCTBKN',
       'HLY_CLOD_PCTOVC', 'years_HLY_CLOD_PCTOVC', 'HLY_HIDX_NORMAL',
       'years_HLY_HIDX_NORMAL', 'HLY_WCHL_NORMAL', 'years_HLY_WCHL_NORMAL',
       'HLY_WIND_AVGSPD', 'years_HLY_WIND_AVGSPD', 'HLY_WIND_PCTCLM',
       'years_HLY_WIND_PCTCLM', 'HLY_WIND_VCTDIR', 'years_HLY_WIND_VCTDIR',
       'HLY_WIND_VCTSPD', 'years_HLY_WIND_VCTSPD', 'HLY_WIND_1STDIR',
       'years_HLY-WIND-1STDIR', 'HLY_WIND_1STPCT', 'years_HLY_WIND_1STPCT',
       'HLY_WIND_2NDDIR', 'years_HLY_WIND_2NDDIR', 'HLY_WIND_2NDPCT',
       'years_HLY_WIND_2NDPCT']
#DROPPING COLUMNS FOR TESTING
df = df.drop(columns=[ 
        'years_HLY_TEMP_NORMAL',
       'years_HLY_TEMP_10PCTL',  'years_HLY_TEMP_90PCTL',
        'years_HLY_DEWP_NORMAL', 
       'years_HLY_DEWPv10PCTL',  'years_HLY_DEWP_90PCTL',
        'years_HLY_PRES_NORMAL', 
       'years_HLY_PRES_10PCTL', 'years_HLY_PRES_90PCTL',
        'years_HLY_CLDH_NORMAL', 
       'years_HLY_HTDH_NORMAL', 'years_HLY_CLOD_PCTCLR',
        'years_HLY_CLOD_PCTFEW', 
       'years_HLY_CLOD_PCTSCT',  'years_HLY_CLOD_PCTBKN',
        'years_HLY_CLOD_PCTOVC', 
       'years_HLY_HIDX_NORMAL',  'years_HLY_WCHL_NORMAL',
        'years_HLY_WIND_AVGSPD', 
       'years_HLY_WIND_PCTCLM',  'years_HLY_WIND_VCTDIR',
        'years_HLY_WIND_VCTSPD', 
       'years_HLY-WIND-1STDIR', 'years_HLY_WIND_1STPCT',
        'years_HLY_WIND_2NDDIR',
       'years_HLY_WIND_2NDPCT'])
# print("-------------------------------NEW COLUMNS-----------------------------")
print(df)
# print('---------------------------------DESCRIBE------------------------------')
# #describe
# print(df.describe())

                date  HLY_TEMP_NORMAL  HLY_TEMP_10PCTL  HLY_TEMP_90PCTL  \
0     01-01T00:00:00             48.8             44.1             54.0   
1     01-01T01:00:00             48.5             43.0             54.0   
2     01-01T02:00:00             47.6             42.1             53.1   
3     01-01T03:00:00             47.2             41.0             54.0   
4     01-01T04:00:00             46.8             39.9             53.1   
...              ...              ...              ...              ...   
8755  12-31T19:00:00             51.5             46.9             55.9   
8756  12-31T20:00:00             51.0             46.0             55.0   
8757  12-31T21:00:00             50.5             46.0             55.0   
8758  12-31T22:00:00             49.8             45.0             54.0   
8759  12-31T23:00:00             49.4             45.0             54.0   

      HLY_DEWP_NORMAL  HLY_DEWP_10PCTL  HLY_DEWP_90PCTL  HLY_PRES_NORMAL  \
0                43.2  

In [43]:
def processNOAAFiles(df, location, fileName, working_directory, destination):

    #drop columns we don't need
    df.drop('STATION', axis=1, inplace=True)
    df.drop('NAME', axis=1, inplace=True)
    df.drop('LATITUDE', axis=1, inplace=True)
    df.drop('LONGITUDE', axis=1, inplace=True)
    df.drop('ELEVATION', axis=1, inplace=True)
    df.drop('month', axis=1, inplace=True)
    df.drop('hour', axis=1, inplace=True)
    df.drop('day', axis=1, inplace=True)

    #checks the flags and removes any rows corresponding with 'M' which means the data is missing
    measurementFlags = list(df.filter(regex='meas'))
    for flag in measurementFlags:
        df = df[~df[flag].str.contains('M')]

    completenessFlags = list(df.filter(regex='meas'))
    for flag in completenessFlags:
        df = df[~df[flag].str.contains('-9999')]

    #get rid of all measurement and completeness flags
    df = df[df.columns.drop(list(df.filter(regex='meas')))]
    df = df[df.columns.drop(list(df.filter(regex='comp')))]

    #rename columns
    df.columns = ['date', 'HLY_TEMP_NORMAL', 'years_HLY_TEMP_NORMAL', 'HLY_TEMP_10PCTL',
       'years_HLY_TEMP_10PCTL', 'HLY_TEMP_90PCTL', 'years_HLY_TEMP_90PCTL',
       'HLY_DEWP_NORMAL', 'years_HLY_DEWP_NORMAL', 'HLY_DEWP_10PCTL',
       'years_HLY_DEWPv10PCTL', 'HLY_DEWP_90PCTL', 'years_HLY_DEWP_90PCTL',
       'HLY_PRES_NORMAL', 'years_HLY_PRES_NORMAL', 'HLY_PRES_10PCTL',
       'years_HLY_PRES_10PCTL', 'HLY_PRES_90PCTL', 'years_HLY_PRES_90PCTL',
       'HLY_CLDH_NORMAL', 'years_HLY_CLDH_NORMAL', 'HLY_HTDH_NORMAL',
       'years_HLY_HTDH_NORMAL', 'HLY_CLOD_PCTCLR', 'years_HLY_CLOD_PCTCLR',
       'HLY_CLOD_PCTFEW', 'years_HLY_CLOD_PCTFEW', 'HLY_CLOD_PCTSCT',
       'years_HLY_CLOD_PCTSCT', 'HLY_CLOD_PCTBKN', 'years_HLY_CLOD_PCTBKN',
       'HLY_CLOD_PCTOVC', 'years_HLY_CLOD_PCTOVC', 'HLY_HIDX_NORMAL',
       'years_HLY_HIDX_NORMAL', 'HLY_WCHL_NORMAL', 'years_HLY_WCHL_NORMAL',
       'HLY_WIND_AVGSPD', 'years_HLY_WIND_AVGSPD', 'HLY_WIND_PCTCLM',
       'years_HLY_WIND_PCTCLM', 'HLY_WIND_VCTDIR', 'years_HLY_WIND_VCTDIR',
       'HLY_WIND_VCTSPD', 'years_HLY_WIND_VCTSPD', 'HLY_WIND_1STDIR',
       'years_HLY-WIND-1STDIR', 'HLY_WIND_1STPCT', 'years_HLY_WIND_1STPCT',
       'HLY_WIND_2NDDIR', 'years_HLY_WIND_2NDDIR', 'HLY_WIND_2NDPCT',
       'years_HLY_WIND_2NDPCT']
    #Dropping these because they don't seem useful
    df = df.drop(columns=[
            'years_HLY_TEMP_NORMAL',
           'years_HLY_TEMP_10PCTL',  'years_HLY_TEMP_90PCTL',
            'years_HLY_DEWP_NORMAL', 
           'years_HLY_DEWPv10PCTL',  'years_HLY_DEWP_90PCTL',
            'years_HLY_PRES_NORMAL', 
           'years_HLY_PRES_10PCTL', 'years_HLY_PRES_90PCTL',
            'years_HLY_CLDH_NORMAL', 
           'years_HLY_HTDH_NORMAL', 'years_HLY_CLOD_PCTCLR',
            'years_HLY_CLOD_PCTFEW', 
           'years_HLY_CLOD_PCTSCT',  'years_HLY_CLOD_PCTBKN',
            'years_HLY_CLOD_PCTOVC', 
           'years_HLY_HIDX_NORMAL',  'years_HLY_WCHL_NORMAL',
            'years_HLY_WIND_AVGSPD', 
           'years_HLY_WIND_PCTCLM',  'years_HLY_WIND_VCTDIR',
            'years_HLY_WIND_VCTSPD', 
           'years_HLY-WIND-1STDIR', 'years_HLY_WIND_1STPCT',
            'years_HLY_WIND_2NDDIR',
           'years_HLY_WIND_2NDPCT'])
    #Rename columns by their location as well
    df.columns = ['date', 'HLY_TEMP_NORMAL' + location, 'HLY_TEMP_10PCTL' + location,
                'HLY_TEMP_90PCTL' + location, 
           'HLY_DEWP_NORMAL' + location, 'HLY_DEWP_10PCTL' + location,
            'HLY_DEWP_90PCTL'  + location,
           'HLY_PRES_NORMAL'  + location, 'HLY_PRES_10PCTL' + location,
            'HLY_PRES_90PCTL' + location, 
           'HLY_CLDH_NORMAL' + location, 'HLY_HTDH_NORMAL' + location,
           'HLY_CLOD_PCTCLR' + location, 
           'HLY_CLOD_PCTFEW' + location, 'HLY_CLOD_PCTSCT' + location,
           'HLY_CLOD_PCTBKN' + location, 
           'HLY_CLOD_PCTOVC' + location, 'HLY_HIDX_NORMAL' + location,
           'HLY_WCHL_NORMAL' + location, 
           'HLY_WIND_AVGSPD' + location,  'HLY_WIND_PCTCLM' + location,
           'HLY_WIND_VCTDIR' + location, 
           'HLY_WIND_VCTSPD' + location, 'HLY_WIND_1STDIR' + location,
            'HLY_WIND_1STPCT' + location,
           'HLY_WIND_2NDDIR' + location,  'HLY_WIND_2NDPCT' + location]
    newCsv = destination + fileName
    df.to_csv(newCsv, encoding='utf-8')

In [44]:
working_directory = '/Users/yukahatori/A_Fairness/NOAA_Data/'
os.chdir(working_directory)
allDataFiles = [f for f in listdir(working_directory) if isfile(join(working_directory, f))] #returns the file names in that directory
files = []
for fileName in allDataFiles:
    print(fileName)
    files.append(fileName)
    df = pd.read_csv(working_directory + fileName)

#00023188 = SD
#00023174 = LA
#00023232 = SAC
#00023152 = burbank
#00023234 = SF
df = pd.read_csv(working_directory + files[0])
processNOAAFiles(df, '_SD', 'SD.csv','/Users/yukahatori/A_Fairness/NOAA_Data/', '/Users/yukahatori/A_Fairness/NOAA_Data_Clean/')
df = pd.read_csv(working_directory + files[1])
processNOAAFiles(df, '_LA', 'LA.csv','/Users/yukahatori/A_Fairness/NOAA_Data/', '/Users/yukahatori/A_Fairness/NOAA_Data_Clean/')
df = pd.read_csv(working_directory + files[2])
processNOAAFiles(df, '_SAC', 'SAC.csv','/Users/yukahatori/A_Fairness/NOAA_Data/', '/Users/yukahatori/A_Fairness/NOAA_Data_Clean/')
df = pd.read_csv(working_directory + files[4])
processNOAAFiles(df, '_SF', 'SF.csv','/Users/yukahatori/A_Fairness/NOAA_Data/', '/Users/yukahatori/A_Fairness/NOAA_Data_Clean/')

USW00023188.csv
USW00023174.csv
USW00023232.csv
USW00023152.csv
USW00023234.csv
